<a href="https://colab.research.google.com/github/stefanziog/Quantized-Deep-Neural-Networks/blob/main/int8/cifar10_int8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import logging
logging.getLogger("tensorflow").setLevel(logging.DEBUG)

import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pylab as plt
from keras.datasets import cifar10
import pathlib

assert float(tf.__version__[:3]) >= 2.3

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# load train and test dataset
def load_dataset():
# load dataset
  (train_images, train_labels), (test_images, test_labels) = cifar10.load_data()
  	
  return train_images, train_labels, test_images, test_labels

# scale pixels
def prep_pixels(train, test):
	# convert from integers to floats
	train_norm = train.astype('float32')
	test_norm = test.astype('float32')
	# normalize to range 0-1
	train_norm = train_norm / 255.0
	test_norm = test_norm / 255.0
	# return normalized images
	return train_norm, test_norm

In [ ]:
#Convert using float fallback quantization
def representative_data_gen():
  for input_value in tf.data.Dataset.from_tensor_slices(train_images).batch(1).take(100):
    # Model has only one input so each data point has one element.
    yield [input_value]

In [ ]:
#CONVERT USING INTEGER ONLY QUANTIZATION
def representative_data_gen():
  for input_value in tf.data.Dataset.from_tensor_slices(train_images).batch(1).take(100):
    yield [input_value]

In [ ]:
#RUN TENSORFLOW MODELS

# Helper function to run inference on a TFLite model
def run_tflite_model(tflite_file, test_image_indices):
  global test_images

  # Initialize the interpreter
  interpreter = tf.lite.Interpreter(model_path=str(tflite_file))
  interpreter.allocate_tensors()

  input_details = interpreter.get_input_details()[0]
  output_details = interpreter.get_output_details()[0]

  predictions = np.zeros((len(test_image_indices),), dtype=int)
  for i, test_image_index in enumerate(test_image_indices):
    test_image = test_images[test_image_index]
    test_label = test_labels[test_image_index]

    # Check if the input type is quantized, then rescale input data to uint8
    if input_details['dtype'] == np.uint8:
      input_scale, input_zero_point = input_details["quantization"]
      test_image = test_image / input_scale + input_zero_point

    test_image = np.expand_dims(test_image, axis=0).astype(input_details["dtype"])
    interpreter.set_tensor(input_details["index"], test_image)
    interpreter.invoke()
    output = interpreter.get_tensor(output_details["index"])[0]

    predictions[i] = output.argmax()

  return predictions

In [ ]:
## Helper function to test the models on one image
def test_model(tflite_file, test_image_index, model_type):
  global test_labels

  predictions = run_tflite_model(tflite_file, [test_image_index])

  plt.imshow(test_images[test_image_index])
  template = model_type + " Model \n True:{true}, Predicted:{predict}"
  _ = plt.title(template.format(true= str(test_labels[test_image_index]), predict=str(predictions[0])))
  plt.grid(False)

  test_model(tflite_model_file, test_image_index, model_type="Float")
  test_model(tflite_model_quant_file, test_image_index, model_type="Quantized")

In [ ]:
#EVALUATE THE MODELS ON ALL IMAGES

# Helper function to evaluate a TFLite model on all images
def evaluate_model(tflite_file, model_type):
  global test_images
  global test_labels

  test_image_indices = range(test_images.shape[0])
  predictions = run_tflite_model(tflite_file, test_image_indices)

  # Htan etsi, opou an eblepes to test_labels, tha katalabaines oti einai 2diastatos pinakas, dld itan [array([[3],8],[8],....], enw eprepe na einai array([3, 8, 8....], dld stin idia diastasi tou prediction. Genika otan exeis themata tsekare diastaseis, times ktl. Px egw apla ekana ena return gia na dw an einai to idio, kai to entopisa.
  # accuracy = (np.sum(test_labels== predictions) * 100) / len(test_images)
  accuracy = (np.sum(test_labels.flatten()== predictions) * 100) / len(test_images)
  print(np.sum(test_labels== predictions))
  print(len(test_labels))
  print(len(test_images))

  print('%s model accuracy is %.2f%% (Number of test samples=%d)' % (
      model_type, accuracy, len(test_images)))
  return predictions, test_labels

In [ ]:
# load dataset
train_images, train_labels, test_images, test_labels = load_dataset()

# prepare pixel data
train_images, test_images = prep_pixels(train_images, test_images)

#load model
model = tf.keras.models.load_model('/content/gdrive/MyDrive/Διπλωματική Κώδικες τελικοί/saved_models/cifar10.h5')

170508288/170498071 [==============================] - 2s 0us/step


In [ ]:
#CONVERT TO TENSORFLOW LITE
converter = tf.lite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpmi3v7tvu/assets


In [ ]:
interpreter = tf.lite.Interpreter(model_content=tflite_model)
input_type = interpreter.get_input_details()[0]['dtype']
print('input: ', input_type)
output_type = interpreter.get_output_details()[0]['dtype']
print('output: ', output_type)

input:  <class 'numpy.float32'>
output:  <class 'numpy.float32'>


In [ ]:
#CONVERT WITH DYNAMIC RANGE
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

tflite_model_quant = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpt2nmmblm/assets


INFO:tensorflow:Assets written to: /tmp/tmpt2nmmblm/assets


In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen

tflite_model_quant = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpy4j2j98x/assets


INFO:tensorflow:Assets written to: /tmp/tmpy4j2j98x/assets


In [ ]:
interpreter = tf.lite.Interpreter(model_content=tflite_model_quant)
input_type = interpreter.get_input_details()[0]['dtype']
print('input: ', input_type)
output_type = interpreter.get_output_details()[0]['dtype']
print('output: ', output_type)

input:  <class 'numpy.float32'>
output:  <class 'numpy.float32'>


In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
# Ensure that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Set the input and output tensors to uint8 (APIs added in r2.3)
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

tflite_model_quant = converter.convert()

INFO:tensorflow:Assets written to: /tmp/tmpkfk7q9ti/assets


INFO:tensorflow:Assets written to: /tmp/tmpkfk7q9ti/assets


In [ ]:
#SHOW TYPE
interpreter = tf.lite.Interpreter(model_content=tflite_model_quant)
input_type = interpreter.get_input_details()[0]['dtype']
print('input: ', input_type)
output_type = interpreter.get_output_details()[0]['dtype']
print('output: ', output_type)

input:  <class 'numpy.uint8'>
output:  <class 'numpy.uint8'>


In [ ]:
tflite_models_dir = pathlib.Path("/tmp/cifar10_int8_tflite_models/")
tflite_models_dir.mkdir(exist_ok=True, parents=True)

# Save the unquantized/float model:
tflite_model_file = tflite_models_dir/"cifar10_int8_model.tflite"
tflite_model_file.write_bytes(tflite_model)

20388436

In [ ]:
# Save the quantized model:
tflite_model_quant_file = tflite_models_dir/"cifar10_int8_model_quant.tflite"
tflite_model_quant_file.write_bytes(tflite_model_quant)

5118072

In [ ]:
# Change this to test a different image
test_image_index = 1
#EVALUATE FLOAT
predictions, test_labels=evaluate_model(tflite_model_file, model_type="Float")

10000000
10000
10000
Float model accuracy is 74.49% (Number of test samples=10000)


In [ ]:
np.sum(predictions==test_labels.flatten())

7449

In [ ]:
test_labels.flatten()

array([3, 8, 8, ..., 5, 1, 7], dtype=uint8)

In [ ]:
#EVALUATE QUANTIZED
evaluate_model(tflite_model_quant_file, model_type="Quantized")

10000000
10000
10000
Quantized model accuracy is 74.45% (Number of test samples=10000)


(array([3, 8, 8, ..., 5, 2, 7]), array([[3],
        [8],
        [8],
        ...,
        [5],
        [1],
        [7]], dtype=uint8))

In [ ]:
ls -lh {tflite_models_dir}

total 25M
-rw-r--r-- 1 root root 4.9M Sep  7 16:03 cifar10_int8_model_quant.tflite
-rw-r--r-- 1 root root  20M Sep  7 16:03 cifar10_int8_model.tflite
